In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import einx
import random
from torch.utils.data import Dataset, DataLoader

def genList(listSize):
        return [(-1 if random.randint(0, 1) else random.random()) for x in range(listSize)]

def genAnswerKey(inputList):
    answerKeyList = []
    for val in inputList:
        numLessThanVal = 0
        for otherVal in inputList:
            if otherVal < val and otherVal > 0:
                numLessThanVal += 1
        answerKeyList.append(numLessThanVal)
    return answerKeyList


class ListDataset(Dataset):
    def __init__(self, listSize, datasetSize):
        self.datasetSize = datasetSize
        self.listSize = listSize
        self.allLists = [genList(self.listSize) for idx in range(self.datasetSize)]
        self.allAnswerKeys = [genAnswerKey(inputList) for inputList in self.allLists]

    def __len__(self):
        return self.datasetSize

    def __getitem__(self, idx):
        return torch.tensor(self.allLists[idx]).float(), torch.tensor(self.allAnswerKeys[idx]).float()

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

basicDataset = ListDataset(3, 100)
sampleLoader = DataLoader(basicDataset, batch_size=1, shuffle=True)
sampleInputTensor, sampleAnswerKey = next(iter(sampleLoader))
print(sampleInputTensor)
print(sampleAnswerKey)

tensor([[-1.0000,  0.7272,  0.9370]])
tensor([[0., 0., 1.]])


In [43]:
import einx.nn.torch as einn
# Transform a (B, L) list to a (B, L, d_emb)
class Stem(nn.Module):
    def __init__(self, embeddingDim):
        super().__init__()
        self.norm = einn.Norm("b [c]")
        self.linearToEmbDim = nn.Linear(1, embeddingDim)
        self.gelu = nn.GELU()
        

    # X expected to be a (B, L) list
    def forward(self, x):
        x = self.norm(x)
        x = einx.rearrange("b c -> b c 1", x)
        x = self.linearToEmbDim(x)
        x = self.gelu(x)
        return x


stem = Stem(4)
sampleEmb = stem(sampleInputTensor)
print(sampleEmb.shape)

torch.Size([1, 3, 4])


In [44]:
import torch


keyDim = 2

sampleQuery = torch.tensor([
    [
        [0., 1],
        [1, 0],
        [1, 1]
    ]
])

sampleKey = torch.tensor([
    [
        [1., 0],
        [0, 1],
        [1, 1]
    ]
])

# Perform attention from all queries to all keys
dotProd = einx.dot("b q [d], b k [d] -> b q k", sampleQuery, sampleKey)
print(dotProd)

softMaxPerQuery = einx.softmax("b q [k]", dotProd / keyDim)
print(softMaxPerQuery)

sampleValDeltas = torch.tensor([
    [
        [2., 0],
        [0, 1],
        [1, 0]
    ]
])

# Each value is associated with the key that outputted that value so we do a matrix multiply across that key dimension to get the weighted sums 
# of each value embedding for each query:
appliedAttentionWeights = einx.dot("b q [k], b [k] h -> b q h", softMaxPerQuery, sampleValDeltas)

print(appliedAttentionWeights)

tensor([[[0., 1., 1.],
         [1., 0., 1.],
         [1., 1., 2.]]])
tensor([[[0.2327, 0.3837, 0.3837],
         [0.3837, 0.2327, 0.3837],
         [0.2741, 0.2741, 0.4519]]])
tensor([[[0.8490, 0.3837],
         [1.1510, 0.2327],
         [1.0000, 0.2741]]])


In [57]:
class AttentionHead(nn.Module):
    def __init__(self, emb_dim, head_size):
        super().__init__()
        self.keyMap = nn.Linear(emb_dim, head_size, bias=False)
        self.queryMap = nn.Linear(emb_dim, head_size, bias=False)
        self.valueDownMap = nn.Linear(emb_dim, head_size, bias=False)
        self.valueUpMap = nn.Linear(head_size, emb_dim, bias=False)

    def forward(self, x):
        # (B, C, emb_dim)
        key = self.keyMap(x) # (B, C, head_size)
        query = self.queryMap(x) # (B, C, head_size)
        valueDown = self.valueDownMap(x) # (B, C, head_size)

        # Self-attention, pairwise dot-prod all queries to all keys then softmax
        attentionDotProd = einx.dot("b q [d], b k [d] -> b q k", query, key) # (B, C, C)
        softMaxPerQuery = einx.softmax("b q [k]", attentionDotProd / np.sqrt(keyDim))
        # Take the weighted sum of the valueDown heads for each channel, where each channel corresponds to 
        appliedAttentionWeights = einx.dot("b q [k], b [k] h -> b q h", softMaxPerQuery, valueDown) # (B C head_size)

        embDelta = self.valueUpMap(appliedAttentionWeights) # (B, C, emb_dim)

        return embDelta, softMaxPerQuery

emb_dim = 4
batchSize = 1
numChannels = 3
head_size = 2

atHead = AttentionHead(emb_dim, head_size)
atHead(sampleEmb)[0]

tensor([[[ 0.0221,  0.0026, -0.0300, -0.0341],
         [ 0.0221,  0.0025, -0.0301, -0.0343],
         [ 0.0221,  0.0024, -0.0301, -0.0344]]], grad_fn=<UnsafeViewBackward0>)

In [64]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        self.head_size = emb_dim // num_heads
        self.attention_heads = nn.ModuleList([AttentionHead(emb_dim, self.head_size) for _ in range(num_heads)])
        
    
    def forward(self, x):
        valueDelta = torch.zeros_like(x)
        perHeadAttentions = []
        for head in self.attention_heads:
            headEmbeddingDelta, headAttention = head(x)
            valueDelta += headEmbeddingDelta
            perHeadAttentions.append(headAttention)
        perHeadAttentions = torch.stack(perHeadAttentions, dim=1)
        return valueDelta, perHeadAttentions

num_heads = 2
multiHeadedAtt = MultiHeadedAttention(emb_dim, num_heads)
multiHeadedAtt(sampleEmb)[0]

tensor([[[-3.0243e-02,  1.0101e-01, -1.3466e-04,  5.1788e-02],
         [-3.0221e-02,  1.0058e-01, -3.9538e-05,  5.1670e-02],
         [-3.0210e-02,  1.0036e-01, -6.3688e-06,  5.1612e-02]]],
       grad_fn=<AddBackward0>)

In [66]:
class TransformerBlock(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        self.attention = MultiHeadedAttention(emb_dim, num_heads)
        self.feedforward_nonlinear = nn.Sequential(
            nn.Linear(emb_dim, emb_dim * 4),
            nn.GELU(),
            nn.Linear(emb_dim * 4, emb_dim)
        )
        self.norm = nn.LayerNorm(emb_dim)
    def forward(self, x):
        attentionOutput, attentionWeights = self.attention(x)
        x = x + attentionOutput
        x = x + self.feedforward_nonlinear(x)
        x = self.norm(x)
        return x, attentionWeights

block = TransformerBlock(emb_dim, num_heads)
block(sampleEmb)[0]

tensor([[[ 1.6476, -0.2719, -0.3245, -1.0512],
         [-1.0225,  1.3004, -0.9211,  0.6432],
         [-1.0777,  0.9780, -0.9188,  1.0184]]],
       grad_fn=<NativeLayerNormBackward0>)

In [73]:
emb_dim = 64
num_heads = 8
num_blocks = 4

class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = Stem(emb_dim)

        self.transformerBlocks = nn.ModuleList([TransformerBlock(emb_dim, num_heads) for _ in range(num_blocks)])

        self.output = nn.Sequential(
            nn.Linear(emb_dim, 1)
        )

    def forward(self, x):
        x = self.stem(x)
        allAttentionWeights = []
        for block in self.transformerBlocks:
            blockOutput, attentionWeights = block(x)
            x = blockOutput
            allAttentionWeights.append(attentionWeights)
        x = self.output(x)
        return x.squeeze(-1), tuple(allAttentionWeights)
        
sampleInputTensor = next(iter(sampleLoader))[0]
sampleTransformer = Transformer()
sampleTransformer(sampleInputTensor)[0].shape

torch.Size([1, 3])

In [70]:

transformer = Transformer()

trainDataset = ListDataset(10, 1000)
trainLoader = DataLoader(trainDataset, batch_size=64, shuffle=True)
testDataset = ListDataset(10, 1000)
testLoader = DataLoader(testDataset, batch_size=256, shuffle=True)

inputTensor, targetTensor = next(iter(trainLoader))
print(inputTensor.shape)
transformer(inputTensor)[0].shape

torch.Size([64, 10])


torch.Size([64, 10])

In [91]:
from torch.optim.lr_scheduler import CosineAnnealingLR

n_epochs = 100
initialLearningRate = 0.003

model = Transformer()

optimizer = torch.optim.Adam(model.parameters(), lr = initialLearningRate)
scheduler = CosineAnnealingLR(optimizer, n_epochs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.L1Loss()

for epoch in range(n_epochs):
    runningLoss = 0
    for inputTensor, targetTensor in trainLoader:
        model.zero_grad()
        pred, _ = model(inputTensor.to(device))
        loss = criterion(pred, targetTensor.to(device))
        loss.backward()
        runningLoss += loss.item() * inputTensor.shape[0]
        optimizer.step()
    scheduler.step()

    runningTestLoss = 0
    for inputTensor, targetTensor in testLoader:
        with torch.no_grad():
            pred, _ = model(inputTensor.to(device))
            loss = criterion(pred, targetTensor.to(device))
            runningTestLoss += loss.item() * inputTensor.shape[0]
    if epoch % 10 == 0:
        print(f"Epoch {epoch:< 4}Train Loss{round(runningLoss / len(trainDataset), 3):< 10}TestLoss{round(runningTestLoss / len(testDataset), 2):< 10}")


Epoch  0  Train Loss 1.367    TestLoss 1.1      
Epoch  10 Train Loss 0.173    TestLoss 0.15     
Epoch  20 Train Loss 0.128    TestLoss 0.13     
Epoch  30 Train Loss 0.077    TestLoss 0.09     
Epoch  40 Train Loss 0.048    TestLoss 0.05     
Epoch  50 Train Loss 0.05     TestLoss 0.06     
Epoch  60 Train Loss 0.034    TestLoss 0.03     
Epoch  70 Train Loss 0.022    TestLoss 0.02     
Epoch  80 Train Loss 0.014    TestLoss 0.02     
Epoch  90 Train Loss 0.01     TestLoss 0.02     


In [ ]:
inp, t = next(iter(trainLoader))
def parseTensor(tensor):
    return '\t'.join([f"{x.item():.2f}" for x in tensor])
print("Input List:")
print(parseTensor(inp[0]))
print("True Target List:")
print(parseTensor(t[0]))
print("Predicted List:")
print(parseTensor(model(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5, -1, -1, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("An In Distribution Input:")
print(parseTensor(inp[0]))
print("True Target:")
print(parseTensor(trueTarget))
print("Predictd List:")
print(parseTensor(model(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.97, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("An Out Of Distribution Input:")
print(parseTensor(inp[0]))
print("True Target:")
print(parseTensor(trueTarget))
print("Predictd List:")
print(parseTensor(model(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.9, 0.95, 0.975, 0.99, 0.995, 0.9975, 0.9999, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("A Pathological Input:")
print(inp[0])
print("True Target:")
print(parseTensor(trueTarget))
print("Predictdd List:")
print(parseTensor(model(inp.to(device))[0]))

In [12]:
from torch.optim.lr_scheduler import CosineAnnealingLR

class DumbModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.dumbLin = nn.Sequential(
            nn.Linear(10, 64),
            nn.GELU(),
            nn.Linear(64,8),
            nn.GELU(),
            nn.Linear(8,10)
        )
    def forward(self, x):
        return self.dumbLin(x)

n_epochs = 500
initialLearningRate = 0.03

dumbModel = DumbModel()

optimizer = torch.optim.Adam(dumbModel.parameters(), lr = initialLearningRate)
scheduler = CosineAnnealingLR(optimizer, n_epochs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dumbModel.to(device)

criterion = nn.L1Loss()

for epoch in range(n_epochs):
    runningLoss = 0
    for inputTensor, targetTensor in trainLoader:
        dumbModel.zero_grad()
        pred = dumbModel(inputTensor.to(device))
        loss = criterion(pred, targetTensor.to(device))
        loss.backward()
        runningLoss += loss.item() * inputTensor.shape[0]
        optimizer.step()
    scheduler.step()

    runningTestLoss = 0
    for inputTensor, targetTensor in testLoader:
        with torch.no_grad():
            pred = dumbModel(inputTensor.to(device))
            loss = criterion(pred, targetTensor.to(device))
            runningTestLoss += loss.item() * inputTensor.shape[0]
    if epoch % 50 == 0:
        print(f"Epoch {epoch:< 7}Train Loss{round(runningLoss / len(trainDataset), 3):< 10}TestLoss{round(runningTestLoss / len(testDataset), 2):< 10}")


Epoch  0     Train Loss 1.091    TestLoss 0.97     
Epoch  50    Train Loss 0.465    TestLoss 0.5      
Epoch  100   Train Loss 0.441    TestLoss 0.5      
Epoch  150   Train Loss 0.42     TestLoss 0.5      
Epoch  200   Train Loss 0.401    TestLoss 0.48     
Epoch  250   Train Loss 0.384    TestLoss 0.48     
Epoch  300   Train Loss 0.377    TestLoss 0.48     
Epoch  350   Train Loss 0.357    TestLoss 0.48     
Epoch  400   Train Loss 0.348    TestLoss 0.48     
Epoch  450   Train Loss 0.342    TestLoss 0.47     


In [13]:
inp, t = next(iter(trainLoader))
def parseTensor(tensor):
    return '\t'.join([f"{x.item():.2f}" for x in tensor])
print("Input List:")
print(parseTensor(inp[0]))
print("True Target List:")
print(parseTensor(t[0]))
print("Predicted List:")
print(parseTensor(dumbModel(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5, -1, -1, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("An In Distribution Input:")
print(parseTensor(inp[0]))
print("True Target:")
print(parseTensor(trueTarget))
print("Predictd List:")
print(parseTensor(dumbModel(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.97, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("An Out Of Distribution Input:")
print(parseTensor(inp[0]))
print("True Target:")
print(parseTensor(trueTarget))
print("Predictd List:")
print(parseTensor(dumbModel(inp.to(device))[0]))


print()
print()
inp = torch.tensor([[0.9, 0.95, 0.975, 0.99, 0.995, 0.9975, 0.9999, -1, -1, -1]])
trueTarget = torch.tensor(genAnswerKey(inp[0]))
print("A Pathological Input:")
print(inp[0])
print("True Target:")
print(parseTensor(trueTarget))
print("Predictdd List:")
print(parseTensor(dumbModel(inp.to(device))[0]))

Input List:
0.50	-1.00	0.63	-1.00	0.63	0.35	-1.00	0.96	-1.00	-1.00
True Target List:
1.00	0.00	2.00	0.00	3.00	0.00	0.00	4.00	0.00	0.00
Predicted List:
0.00	-0.00	1.89	-0.00	-0.00	1.11	-0.00	3.74	-0.00	0.00


An In Distribution Input:
0.10	0.20	0.30	0.40	0.50	-1.00	-1.00	-1.00	-1.00	-1.00
True Target:
0.00	1.00	2.00	3.00	4.00	0.00	0.00	0.00	0.00	0.00
Predictd List:
0.00	0.86	1.64	2.62	0.00	-0.00	0.00	-0.11	0.00	-0.00


An Out Of Distribution Input:
0.50	0.60	0.70	0.80	0.90	0.95	0.97	-1.00	-1.00	-1.00
True Target:
0.00	1.00	2.00	3.00	4.00	5.00	6.00	0.00	0.00	0.00
Predictd List:
0.00	0.30	3.85	4.11	0.00	3.84	4.68	0.01	0.00	0.00


A Pathological Input:
tensor([ 0.9000,  0.9500,  0.9750,  0.9900,  0.9950,  0.9975,  0.9999, -1.0000,
        -1.0000, -1.0000])
True Target:
0.00	1.00	2.00	3.00	4.00	5.00	6.00	0.00	0.00	0.00
Predictdd List:
0.00	1.46	4.72	3.51	0.00	1.98	3.52	0.01	0.00	0.00


In [97]:
model.eval()
sampleInputs, sampleOutputs = next(iter(trainLoader))
sampleInputs = sampleInputs[0]
attention = model(sampleInputs.to(device).unsqueeze(0))[-1]

In [98]:
#!pip install bertviz
from bertviz import head_view, model_view

head_view(attention, [str(x) for x in sampleInputs])


<IPython.core.display.Javascript object>